# Nicolas Dorval

<h1 style="text-align:center">Travail pratique numérique en thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 2 : Modèle de Drude</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

- Pierre-Olivier Desrosiers<br>
- Nicolas Dorval<br>
- Gérémy Michaud<br>
- Félix-Antoine Dupuis

# Introduction # 
Cette deuxième partie du travail implémente une simulation 2D du modèle de Drude pour décrire le mouvement des électrons libres et indépendants dans les métaux. Des sphères dures sont encore utilisées pour représenter les particules, mais maintenant de deux types différents afin de différencier les coeurs ioniques immobiles des électrons sur leurs trajectoires balistiques. Les collisions entre les deux doivent donc être inélastiques. Les questions sur cette simulation, d'abord de cinétique puis de dynamique des électrons en présence d'une force externe appliquée au cristal, vérifieront quelques résultats analytiques du modèle de Drude obtenus en classe et/ou dans le manuel de cours Ashcroft/Mermin.

- **La remise du présent _Jupyter Notebook_ ET celui de la 1<sup>re</sup> partie, ainsi que tout autre fichier de code produit, se fait sur Gradescope en n'oubliant pas d'y indiquer tous les membres de votre équipe.**

In [9]:
from numpy import *
import random
from vpython import *
from IPython.display import display, Math
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import maxwell

<IPython.core.display.Javascript object>

Faites une copie du script `TDS-2Dsimulation_HXX.py` et modifiez-le pour obtenir une simulation cinétique d'un gaz d'électrons libres dans la matière cristalline selon le modèle de Drude. Spécifiquement selon les pp.4-6 du manuel Ashcroft/Mermin,
1. ajoutez un deuxième type de sphères fixes réparties périodiquement dans la boîte d'animation: celles-ci représenteront les coeurs ioniques,
2. éliminez les collisions entre les sphères mobiles qui représentent maintenant les électrons de conduction indépendants et libres,
3. en faisant appel à la température du gaz, ajoutez des collisions inélastiques entre les électrons libres et les coeurs ioniques fixes. La quantité de mouvement $\vec{p}$ n'est alors PAS conservée et il faut appliquer les hypothèses spécifiques du modèle de Drude à la sortie de chaque collision, notamment: 
- la direction de $\vec{p}$ doit être aléatoire,
- la norme $||\vec{p}||$ est déterminée par la distribution de Maxwell-Boltzmann.

### Votre simulation ###

VII. _(25 points)_ &mdash; Écrivez un appel de votre nouveau script pour l'exécuter avec la cellule suivante:

In [10]:
#Code de GM (test4) modifié légèrement pour la question 10

# Created on Fri Feb 5 15:40:27 2021
# GlowScript 3.0 VPython

# Hard-sphere gas.

# Bruce Sherwood
# Claudine Allen

def DrudeModelSimulator(iterations, champ):
    # Simulation parameters
    N_electrons = 200
    N_cores = 64  # It is easier when you choose a squared number for the number of cores
    dt = 1E-7  # Increase the time increment if you dare! Brace yourself for a spectacular display of fireworks.
    electron2follow = 0

    # Physical constants
    mass_electron = 9E-31
    R_electron = 0.01  # wildly exaggerated size of an atom
    R_core = 0.025
    k = 1.4E-23
    T = 300
    q = 1.602E-19

    start_time = time.time()

    # CANEVAS DE FOND
    L = 1
    animation = canvas(width=750, height=500, align='center')
    animation.range = L
    animation.title = 'Simulation du modèle de Drude'
    s = """
    Simulation de particules modélisées en sphères pour représenter le modèle de Drude.
    Les sphères représentent les électrons mobiles (gris) et les coeurs ioniques fixes (rouges).
    """
    animation.caption = s
    d = L / 2 + max(R_core, R_electron)
    cadre = curve(color=color.blue, radius=0.005)
    cadre.append([vector(-d, -d, 0), vector(d, -d, 0), vector(d, d, 0),
                vector(-d, d, 0), vector(-d, -d, 0)])

    # Initialize electrons
    Electrons = []
    p = []
    apos = []
    pavg = sqrt(2 * mass_electron * 1.5 * k * T)

    for i in range(N_electrons):
        x = L * random() - L / 2
        y = L * random() - L / 2
        z = 0
        if i == electron2follow:
            Electrons.append(simple_sphere(pos=vector(x, y, z), radius=0.02, color=color.magenta))
        else:
            Electrons.append(simple_sphere(pos=vector(x, y, z), radius=R_electron, color=color.gray(0.7)))
        apos.append(vec(x, y, z))

        phi = 2 * pi * random()
        px = pavg * cos(phi)
        py = pavg * sin(phi)
        pz = 0
        p.append(vector(px, py, pz))

    # Initialize ion cores
    Cores = []
    cpos = []
    ion_core_spacing = L / (sqrt(N_cores) - 1)

    for i in range(int(sqrt(N_cores))):
        for j in range(int(sqrt(N_cores))):
            # Create a 2D grid of ion cores with periodic arrangement within the square
            x = i * ion_core_spacing - L / 2
            y = j * ion_core_spacing - L / 2
            z = 0
            Cores.append(simple_sphere(pos=vector(x, y, z), radius=R_core, color=color.red))
            cpos.append(vec(x, y, z))

    def checkCollisions():
        """Check for collisions.

        Returns:
            list: List of collisions
        """
        hitlist = []
        r2 = (R_electron + R_core) ** 2

        for i in range(len(cpos)):
            ci = cpos[i]
            for j in range(len(apos)):
                aj = apos[j]
                # Does not care for electron-electron collisions
                dr = ci - aj
                if mag2(dr) < r2:
                    hitlist.append([i, j])

        return hitlist

    pavg_list = []
    posavg_y_list = []
    posavg_x_list = []

    # Main simulation
    for i in range(iterations):
        rate(300)

        pavg = np.mean([nb_p.mag for nb_p in p])
        posavg_y = np.mean([pos.y for pos in apos])
        posavg_x = np.mean([pos.x for pos in apos])

        # Move electrons
        for i in range(N_electrons):
            vitesse = p[i] / mass_electron
            deltax = vitesse * dt
            Electrons[i].pos = apos[i] = apos[i] + deltax

        # Conserve momentum with wall collisions
        for i in range(N_electrons):
            loc = apos[i]
            if abs(loc.x) > L / 2:
                p[i].x = abs(p[i].x) if loc.x < 0 else -abs(p[i].x)
            if abs(loc.y) > L / 2:
                p[i].y = abs(p[i].y) if loc.y < 0 else -abs(p[i].y)
                #p[i].y += q * champ * dt

        hitlist = checkCollisions()

        pavg_list.append(pavg)
        posavg_y_list.append(posavg_y)
        posavg_x_list.append(posavg_x)

        temperature = pavg**2 / (3 * k * mass_electron)
        # Handle electron-core collisions
        for ij in hitlist:
            i = ij[1]
            j = ij[0]
            posi = apos[i]
            posj = cpos[j]
            vi = p[i] / mass_electron
            rrel = hat(posi - posj)
            vrel = vi

            #dx = dot(posi-posj, vrel.hat)
            #dy = cross(posi-posj, vrel.hat).mag
            #alpha = asin(dy/(R_electron+R_core))
            #d = (R_electron+R_core)*cos(alpha)-dx
            deltat = d / vrel.mag

            theta = 2 * pi * random()
            norm_p = mass_electron * maxwell.rvs(scale=sqrt(k * temperature / mass_electron))
            p[i] = norm_p * hat(rotate(rrel, angle=theta))
            apos[i] = posi + (posi - posj) * deltat

    return [pavg_list, posavg_y_list, posavg_x_list]


In [ ]:
nbre_iter = 700
liste_p_avg, liste_pos_avg_y, liste_pos_avg_x = DrudeModelSimulator(nbre_iter, 0)

### Question statistique ###

VIII. _(5 points)_ &mdash; Vérifiez numériquement et graphiquement que l'amortissement de la quantité de mouvement moyenne des électrons suit l'équation exponentielle dérivée analytiquement en classe, soit $\langle p(t)\rangle =\langle p(t_0)\rangle \,e^{-t/\tau}$, et comparez-y le comportement d'un seul électron.

In [ ]:
iterations = np.linspace(0, nbre_iter, nbre_iter)
p_init = liste_p_avg[0]
modele = lambda t, tau: p_init*np.exp(-t/tau)
popt, pcov = curve_fit(modele, iterations, liste_p_avg)
plt.figure(figsize=(8, 6))
plt.plot(iterations, liste_p_avg, label="Simulation")
plt.plot(iterations, modele(iterations, popt), label="Modèle")
plt.legend()
plt.title("Quantité de mouvement d'un électron")
plt.xlabel("Itérations")
plt.ylabel("p(t) [kg m/s]")
plt.show()

### Dynamique sous l'effet d'une force externe ###

IX. _(10 points)_ &mdash; Pour passer de la cinétique à la dynamique des électrons libres, modifiez votre code de simulation en ajoutant une fonction qui applique un champ électrique uniforme. Celui-ci devra être de module ajustable et perpendiculaire à deux des côtés de la boîte. À chaque pas de temps $\mathrm{d}t$ sans collision, les électrons devront donc accélérer d'un incrément $\mathrm{d}p_x$ dicté par la force de Coulomb à leur position.

Copiez le code de votre fonction dans la cellule qui suit en n'oubliant pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

In [ ]:
# Modification de la simulation afin de prendre en compte Δpx, mais les électrons quittent le modèle vers les x+

# Created on Fri Feb 5 15:40:27 2021
# GlowScript 3.0 VPython

# Hard-sphere gas.

# Bruce Sherwood
# Claudine Allen

def DrudeModelSimulator(iterations, champ):
    # Simulation parameters
    N_electrons = 200
    N_cores = 64  # It is easier when you choose a squared number for the number of cores
    dt = 1E-7  # Increase the time increment if you dare! Brace yourself for a spectacular display of fireworks.
    electron2follow = 0

    # Physical constants
    mass_electron = 9E-31
    R_electron = 0.01  # wildly exaggerated size of an atom
    R_core = 0.025
    k = 1.4E-23
    T = 300
    q = 1.602E-19

    start_time = time.time()

    # CANEVAS DE FOND
    L = 1
    animation = canvas(width=750, height=500, align='center')
    animation.range = L
    animation.title = 'Simulation du modèle de Drude'
    s = """
    Simulation de particules modélisées en sphères pour représenter le modèle de Drude.
    Les sphères représentent les électrons mobiles (gris) et les coeurs ioniques fixes (rouges).
    """
    animation.caption = s
    d = L / 2 + max(R_core, R_electron)
    cadre = curve(color=color.blue, radius=0.005)
    cadre.append([vector(-d, -d, 0), vector(d, -d, 0), vector(d, d, 0),
                vector(-d, d, 0), vector(-d, -d, 0)])

    # Initialize electrons
    Electrons = []
    p = []
    apos = []
    pavg = sqrt(2 * mass_electron * 1.5 * k * T)

    for i in range(N_electrons):
        x = L * random() - L / 2
        y = L * random() - L / 2
        z = 0
        if i == electron2follow:
            Electrons.append(simple_sphere(pos=vector(x, y, z), radius=0.02, color=color.magenta))
        else:
            Electrons.append(simple_sphere(pos=vector(x, y, z), radius=R_electron, color=color.gray(0.7)))
        apos.append(vec(x, y, z))

        phi = 2 * pi * random()
        px = pavg * cos(phi)
        py = pavg * sin(phi)
        pz = 0
        p.append(vector(px, py, pz))

    # Initialize ion cores
    Cores = []
    cpos = []
    ion_core_spacing = L / (sqrt(N_cores) - 1)

    for i in range(int(sqrt(N_cores))):
        for j in range(int(sqrt(N_cores))):
            # Create a 2D grid of ion cores with periodic arrangement within the square
            x = i * ion_core_spacing - L / 2
            y = j * ion_core_spacing - L / 2
            z = 0
            Cores.append(simple_sphere(pos=vector(x, y, z), radius=R_core, color=color.red))
            cpos.append(vec(x, y, z))

    def checkCollisions():
        """Check for collisions.

        Returns:
            list: List of collisions
        """
        hitlist = []
        r2 = (R_electron + R_core) ** 2

        for i in range(len(cpos)):
            ci = cpos[i]
            for j in range(len(apos)):
                aj = apos[j]
                # Does not care for electron-electron collisions
                dr = ci - aj
                if mag2(dr) < r2:
                    hitlist.append([i, j])

        return hitlist

    pavg_list = []
    posavg_y_list = []
    posavg_x_list = []

    # Main simulation
    for i in range(iterations):
        rate(300)

        pavg = np.mean([nb_p.mag for nb_p in p])
        posavg_y = np.mean([pos.y for pos in apos])
        posavg_x = np.mean([pos.x for pos in apos])

        # Move electrons
        for i in range(N_electrons):
            vitessex = (p[i].x + q *  champ * dt) / mass_electron
            vitessey = p[i].y / mass_electron
            deltax = vitessex * dt
            deltay = vitessey * dt
            Electrons[i].pos = apos[i] = apos[i] + vector(deltax, deltay, 0)

        # Conserve momentum with wall collisions
        for i in range(N_electrons):
            loc = apos[i]
            if abs(loc.x) > L / 2:
                p[i].x = abs(p[i].x) if loc.x < 0 else -abs(p[i].x)
            if abs(loc.y) > L / 2:
                p[i].y = abs(p[i].y) if loc.y < 0 else -abs(p[i].y)
                #p[i].y += q * champ * dt

        hitlist = checkCollisions()

        pavg_list.append(pavg)
        posavg_y_list.append(posavg_y)
        posavg_x_list.append(posavg_x)

        temperature = pavg**2 / (3 * k * mass_electron)
        # Handle electron-core collisions
        for ij in hitlist:
            i = ij[1]
            j = ij[0]
            posi = apos[i]
            posj = cpos[j]
            vi = p[i] / mass_electron
            rrel = hat(posi - posj)
            vrel = vi

            #dx = dot(posi-posj, vrel.hat)
            #dy = cross(posi-posj, vrel.hat).mag
            #alpha = asin(dy/(R_electron+R_core))
            #d = (R_electron+R_core)*cos(alpha)-dx
            deltat = d / vrel.mag

            theta = 2 * pi * random()
            norm_p = mass_electron * maxwell.rvs(scale=sqrt(k * temperature / mass_electron))
            p[i] = norm_p * hat(rotate(rrel, angle=theta))
            apos[i] = posi + (posi - posj) * deltat

    return [pavg_list, posavg_y_list, posavg_x_list]


### Question statistique ###

X. _(5 points)_ &mdash; Pour quelques différents modules de champ électrique, présentez graphiquement l'évolution de la position moyenne des électrons en fonction du temps pour ses deux composantes parallèle et perpendiculaire au champ.

In [ ]:
liste_p_avg_1, liste_pos_avg_y_1, liste_pos_avg_x_1 = DrudeModelSimulator(nbre_iter, 0)
liste_p_avg_2, liste_pos_avg_y_2, liste_pos_avg_x_2 = DrudeModelSimulator(nbre_iter, 1)
liste_p_avg_3, liste_pos_avg_y_3, liste_pos_avg_x_3 = DrudeModelSimulator(nbre_iter, 2)

iterations = np.linspace(0, nbre_iter, nbre_iter)

fig, axs = plt.subplots(2)
plt.subplots_adjust(left=0.15, bottom=0.1, right=0.95, top=0.9, wspace=0.4, hspace=0.5)

axs[0].plot(iterations, liste_pos_avg_y_1)
axs[0].plot(iterations, liste_pos_avg_y_2)
axs[0].plot(iterations, liste_pos_avg_y_3)

axs[1].plot(iterations, liste_pos_avg_x_1)
axs[1].plot(iterations, liste_pos_avg_x_2)
axs[1].plot(iterations, liste_pos_avg_x_3)

axs[0].legend(["Champ électrique nul", "Champ électrique de module 1", "Champ électrique de module 2"], loc="upper left")
axs[0].set_xlabel("Itérations")
axs[0].set_ylabel("Position moyenne")
axs[0].title.set_text("Position moyenne parallèle (y) au champ électrique")

axs[1].legend(["Champ électrique nul", "Champ électrique de module 1", "Champ électrique de module 2"], loc="upper left")
axs[1].set_xlabel("Itérations")
axs[1].set_ylabel("Position moyenne")
axs[1].title.set_text("Position moyenne parallèle (x) au champ électrique")
plt.show()

# Médiagraphie #
- P. Drude, _Zur Elektronentheorie der Metalle; I Teil_, Annalen der Physik **306**(3), pp.566–613 (1900) https://doi.org/10.1002/andp.19003060312
- P. Drude, _Zur Elektronentheorie der Metalle; II Teil. Galvanomagnetische und Thermomagnetische Effecte_, Annalen der Physik **308**(11), pp.369–402 (1900) https://doi.org/10.1002/andp.19003081102
- P. Drude, _Zur Elektronentheorie der Metalle; Berichtigung_, Annalen der Physik **312**(3), pp.687–692 (1902) https://doi.org/10.1002/andp.19023120312
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies I_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.438-453 (1905) https://dwc.knaw.nl/DL/publications/PU00013989.pdf
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies II_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.585-593 (1905) https://dwc.knaw.nl/DL/publications/PU00014010.pdf
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies III_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.684-691 (1905) https://dwc.knaw.nl/DL/publications/PU00014024.pdf
- La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour faciliter la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le code adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.